# Segmenting and Clustering Neighborhoods in Toronto 
### Peer-graded assignment in Applied Data Science Capstone by IBM/Coursera 

#### *Submitted by Jessiedee Mark B. Gingo*

## Table of contents
* [Installing Required Libraries](#installaiong)
* [Webscraping](#webscraping)
* [Data Cleaning](#cleaning)
* [Dataframe](#dataframe)


In this assignment, we will explore and cluster the neighborhoods in Toronto. The dataset is not readily available and we will be scraping it on Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

## Installing Required Libraries <a name="installation"></a>

For webscraping, we will be utilizing **BeautifulSoup** package. Our chosen parser is **lxml** and we will install **html5** as well to parse data from a website without any problem.

In [2]:
!pip install BeautifulSoup4
!pip install lxml
!pip install html5

     |████████████████████████████████| 112kB 11.4MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 29.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 8.4MB/s eta 0:00:01


We will import required libraries. BeautifulSoup for webscraping, requests for GET request to a webpage, and pandas so we can prepare the data.

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

The following code is executed to source and html file is converted to text file. We then create the **BeautifulSoup** object, passing the source.

## Webscraping <a name="webscraping"></a>

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')


We will now define our column names. By inspecting the website, headers of the table is under **'th'** tag. The **'th'** tag is passed to the **find_all** method to create a list under **'th'** tag. Slicing the first 3 gives us **'Postcode', 'Borough'**, and **'Neighbourhood'.**

In [5]:
columns = [] # Column values

for header in soup.find_all('th')[0:3]:
    header = header.text
    columns.append(header)

columns[-1] = columns[-1].strip() # Removing the \n in the last item

Preparing the data. The rows list is created to contain the information. From the website, the data are located under **'tr'** tag. Passing to the **find_all** method. 

In [6]:
# Rows values
rows = [] # Rows value container

for data in soup.find_all('tr')[1:]:
    data = data.text
    data = data.split('\n')
    rows.append(data)

rows = rows[0:-5] # Remove unwanted information
    
for i in range(len(rows)): # Clean data by removing the whitespace in the list
    rows[i] = rows[i][1:4]

rows

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', 'Downtown Toronto', "Queen's Park"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', "Queen's Park", 'Not assigned'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Et

## Data Cleaning <a name="cleaning"></a>

The columns and rows are now established and the dataframe is will be created.

In [7]:
df = pd.DataFrame(data=rows, columns=columns) 
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Items that is 'Not assigned' in  **Borough** column is dropped.

In [8]:
df = df[df.Borough != 'Not assigned'].reset_index()
df = df.drop(['index'], axis=1) # redundant index column is removed

**Neighbourhood** with the same **Postalcode** is grouped and concatenated.

In [9]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


The **Neighbourhood** with 'Not assigned' value will be replaced by the value in **Borough** column.

In [10]:
df_not_assigned = df[df['Neighbourhood'].str.contains("Not assigned")] # Extracting the dataframe whose Neighbourhood is 'Not assigned'

for i in list(df_not_assigned['Borough']): # 'Not assigned' value is replaced
    df = df.replace('Not assigned', i)

## Dataframe <a name="dataframe"></a>

First 12 items in the dataframe are shown below.

In [11]:
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


The shape of the dataframe is shown.

In [12]:
df.shape

(103, 3)

## Coordinates

Importing the required libraries.

The **latitude** and **longitude** of every Postcode will be obtained by using Geocoder library. Since the Geocoder package is not working properly, then the downloaded csv file of latitude and longitude will be used.

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [14]:
!pip install geocoder

     |████████████████████████████████| 102kB 5.7MB/s ta 0:00:011
     |████████████████████████████████| 829kB 19.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 18.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
for postal_code in df['Postcode']:
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    df['Latitude'] = lat_lng_coords[0]
    df['Longitude'] = lat_lng_coords[1]

The Geocoder is not working properly. Csv file of the coordinates will be downloaded and converted to dataframe as **df_loc.**

In [16]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [17]:
df_loc = pd.read_csv('Geospatial_Coordinates.csv')

We will reference the **Postal Codes** in our dataset to add latitude and longitude in our **df** dataframe.

In [19]:
for post_code in df_loc['Postal Code']:
    post_code == df['Postcode']
    df['Latitude'] = df_loc['Latitude']
    df['Longitude'] = df_loc['Longitude']

In [20]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Mapping Toronto Postal Code

We will now map all places in Toronto based on the Postal Code.

In [21]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [22]:
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], 
                        radius=5, 
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_of_toronto)
    
map_of_toronto

## Selecting Neighbourhood under the Borough that named Toronto 

We will analyse the Neighbourhood under a Borough that named Toronto. 

In [26]:
borough_named_toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
borough_named_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In this section, we will segment and cluster the neighbourhood under a borough that has a name Toronto. We will use FourSquare API to extract venue information and category to to proceed in Analysis.

### FourSquare API credential

In [24]:
CLIENT_ID = 'IIIODSQOYJSCWYL5N0DPK2HHWKSCHFSMAPV2AWSQ0KSURL21' # your Foursquare ID
CLIENT_SECRET = 'VBF3SZ4LR4KHHEU10SU4NQIATY42ETEKAJBEMOTVKT3XXWXG' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IIIODSQOYJSCWYL5N0DPK2HHWKSCHFSMAPV2AWSQ0KSURL21
CLIENT_SECRET:VBF3SZ4LR4KHHEU10SU4NQIATY42ETEKAJBEMOTVKT3XXWXG


Getting nearby venues using FourSquare API

In [28]:
LIMIT = 100


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=borough_named_toronto['Neighbourhood'],
                                   latitudes=borough_named_toronto['Latitude'],
                                   longitudes=borough_named_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [32]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
